In [ ]:
from astropy.io import fits

In [ ]:
s = "DimlessF_a9.98200e-01.e_8.84e-01.Mdl_5.00e+00.a13_1.00e+00.a22_2.00e+00.a52_5.00e+00.dat"
a = s[10:].split(".")
a = [a[0]+"."+a[1], a[2].split("_")[1]+"."+a[3], a[4].split("_")[1]+"."+a[5], a[6].split("_")[1]+"."+a[7], a[8].split("_")[1]+"."+a[9]]
print(a)

In [ ]:
import os
files = os.listdir("data_Mdl")
print(len(files))
files = os.listdir("data_a13")
print(len(files))
files = os.listdir("data_a22")
print(len(files))
files = os.listdir("data_epsi3")
print(len(files))

In [ ]:
def parse_filename(filename):
    a = filename[10:].split(".")
    a = [a[0]+"."+a[1], a[2].split("_")[1]+"."+a[3], a[4].split("_")[1]+"."+a[5], a[6].split("_")[1]+"."+a[7], a[8].split("_")[1]+"."+a[9]]
    return float(a[0]), float(a[1]), float(a[2]), float(a[3]), float(a[4])
parse_filename("DimlessF_a9.98200e-01.e_8.84e-01.Mdl_5.00e+00.a13_1.00e+00.a22_2.00e+00.a52_5.00e+00.dat")

def add_var(dictt, key, item):
    if key in dictt.keys():
        if item not in dictt[key]:
            dictt[key].append(item)
    else:
        dictt[key] = [item]
    return dictt

def add_to_dict(file, dictt, dtype):
    a0,a1,a2,a3,a4 = parse_filename(file)
    
    if dtype == 1:
        if a0 in dictt.keys():
            if a1 not in dictt[a0].keys():
                dictt[a0][a1]=file
        else:
            dictt[a0] = {a1:file}
            
    if dtype == 2:
        if a0 in dictt.keys():
            if a2 not in dictt[a0].keys():
                dictt[a0][a2]=file
        else:
            dictt[a0] = {a2:file}
            
    if dtype == 3:
        if a0 in dictt.keys():
            if a3 not in dictt[a0].keys():
                dictt[a0][a3]=file
        else:
            dictt[a0] = {a3:file}
            
    if dtype == 4:
        if a0 in dictt.keys():
            if a4 not in dictt[a0].keys():
                dictt[a0][a4]=file
        else:
            dictt[a0] = {a4:file}
    return dictt
    

    
def parse_file(file):
    r, f = [], []
    readMe = open(file,'r').readlines()
    for line in readMe:
        r.append(float(line.split()[0]))
        f.append(float(line.split()[1]))
    return r, f

def sort_dict(dictt):
    b = {k: dictt[k] for k in sorted(dictt)}
    for key in b.keys():
        c = b[key]
        c = {k: c[k] for k in sorted(c)}
        b[key]=c
    return b

def check_dict(dictt):
    if len(dictt) != 30:
        print("Length of dictionary is less than 30")
    for key1 in dictt.keys():
        if len(dictt[key1]) != 30:
            print("Length of dictionary item %f is less than 30"%(key1))
            print(len(dictt[key1]))

            
def print_dict(dictt):
    i=1
    for key1 in dictt.keys():
        j=1
        for key2 in dictt[key1].keys():
            print(str(i)+"_"+str(j)+" "+dictt[key1][key2])
            j+=1
        i+=1

In [ ]:
dirs = ["data_epsi3","data_a13", "data_a22", "data_Mdl"]
a_a13, a_a22, a_epsi3, a_Mdl = {},{},{},{}
    

for i in range(len(dirs)):
    d = dirs[i]
    dtype = i+1
    files = os.listdir(d)
    
    if dtype == 1:
        for file in files:
            a_epsi3 = add_to_dict(file,a_epsi3,dtype)
    if dtype == 2:
        for file in files:
            a_a13 = add_to_dict(file,a_a13,dtype)
    if dtype == 3:
        for file in files:
            a_a22 = add_to_dict(file,a_a22,dtype)
    if dtype == 4:
        for file in files:
            a_Mdl = add_to_dict(file,a_Mdl,dtype)


a_a13=sort_dict(a_a13)
a_a22=sort_dict(a_a22)
a_epsi3=sort_dict(a_epsi3)
a_Mdl=sort_dict(a_Mdl)
print_dict(a_a13)
check_dict(a_a13)
print_dict(a_Mdl)
check_dict(a_Mdl)

In [ ]:
spin, a13, Cols = [], [],[]
i = 1
for key1 in a_a13.keys():
    spin.append(key1)
    j = 1
    tmp=[]
    for key2 in a_a13[key1].keys():
        tmp.append(key2)
        _, f = parse_file('data_a13/'+a_a13[key1][key2])
        nm = "F_"+str(i)+"_"+str(j)
        Cols.append(fits.Column(name=nm, format='1E',array=f))
        j+=1
    i+=1
    a13.append(tmp)


prihdr = fits.Header()
prihdr['INFO'] = "Dimensionless F function for alpha13"
prihdr['AUTHORS'] = 'Fudan Gravity Group'
prihdr['COMMENTS'] = "Makes a file containing tabulated Dimensionless F function values."
prihdu = fits.PrimaryHDU(header=prihdr)
aux=[prihdu]

cola = fits.Column(name='a', format='1E',array=spin)
colb = fits.Column(name='a13', format='30E',array=a13)
cols = fits.ColDefs([cola, colb])

tbhdu = fits.BinTableHDU.from_columns(cols)
aux.append(tbhdu)
tbhdu = fits.BinTableHDU.from_columns(Cols)
aux.append(tbhdu)

print ("len of aux -- " + str(len(aux)))

thdulist = fits.HDUList(aux)
thdulist.writeto('dimlessF_a13.fits')
thdulist.close()